In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install pymorphy2[fast]
!pip3 install smart_open
!pip3 install h5py
clear_output()

In [0]:
%load_ext autoreload
%load_ext pycodestyle_magic

import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from warnings import filterwarnings
from scipy.sparse import csr_matrix
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [0]:
filterwarnings("ignore")

## word2vec + fasttext

загрузка модели

In [0]:
# если модель без тэгов
model_file = "./fasttext/model.model"

model = Word2Vec.load(model_file)

In [0]:
# если модель с POS-тэггингом
model_file = "./fasttext/model.model"

model = KeyedVectors.load(model_file)

проверка наличия слова в словаре

In [0]:
lemma = "черепаха"
lemma in model.vocab

True

получение вектора слова

In [0]:
model.wv[lemma]

In [0]:
model[lemma].shape

(300,)

получение вектора документа

In [0]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный_ADJ', 'замок_NOUN']

# создаем маски для векторов 
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model.wv:
        lemmas_vectors[idx] = model.wv[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

In [0]:
!wget https://www.dropbox.com/s/cfjv7galp6ajyr0/quora_question_pairs_rus.csv?dl=0 -O quora_question_pairs_rus.csv
clear_output()

In [6]:
df = pd.read_csv("quora_question_pairs_rus.csv")
df = df.dropna()
df.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
queries = list(set(df["question1"]))
query_idx = {queries[i]: i for i in range(len(queries))}
docs = list(set(df["question2"]))
doc_idx = {docs[i]: i for i in range(len(docs))}

In [0]:
df_dup = df[df["is_duplicate"] == 1]
row_ind = df_dup["question1"].apply(lambda x: query_idx[x])
col_ind = df_dup["question2"].apply(lambda x: doc_idx[x])
dup_matrix = csr_matrix((np.ones(df_dup.shape[0]), (row_ind, col_ind)),
                        shape=(len(query_idx), len(doc_idx)))

In [0]:
def accuracy(search_engine, queries, dup_matrix, test_size=10000):
    true_results = 0
    all_results = 0
    test = np.random.choice(queries, size=test_size)
    for query in test:
        if dup_matrix[query_idx[query], ].sum():
            all_results += 1
            results = search_engine.search(query)
            for result, score in results:
                if dup_matrix[query_idx[query], doc_idx[result]]:
                    true_results += 1
                    break
    return true_results / all_results

In [0]:
m = MorphAnalyzer()


def lemmatize(text):
    return [m.parse(word)[0].normal_form 
            for word in simple_word_tokenize(text)]

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   

##Fasttext

In [0]:
!mkdir fasttext
!wget http://vectors.nlpl.eu/repository/11/181.zip -O fasttext.zip
!unzip fasttext.zip -d fasttext
!rm fasttext.zip
clear_output()

In [0]:
class SearchFasttext():
    def __init__(self, data, model_file):
        self.texts = np.array(data)
        self.model = KeyedVectors.load(model_file)
        self.vec = np.zeros((len(self.texts), self.model.vector_size))
        for i in range(len(self.texts)):
            lemmas = lemmatize(self.texts[i])
            lemmas_vectors = np.zeros((len(lemmas), self.model.vector_size))
            for idx, lemma in enumerate(lemmas):
                if lemma in self.model.vocab:
                    lemmas_vectors[idx] = self.model[lemma]
            if lemmas_vectors.shape[0] is not 0:
                self.vec[i] = np.mean(lemmas_vectors, axis=0)            

    def search(self, query, n=5):
        lemmas = lemmatize(query)
        query_vec = np.zeros((self.model.vector_size, ))
        lemmas_vectors = np.zeros((len(lemmas), self.model.vector_size))
        for idx, lemma in enumerate(lemmas):
            if lemma in self.model.vocab:
                lemmas_vectors[idx] = self.model[lemma]
        if lemmas_vectors.shape[0] is not 0:
            query_vec = np.mean(lemmas_vectors, axis=0)
        query_vec = np.transpose(query_vec)
        result = np.matmul(self.vec, query_vec)
        indices = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[indices], result[indices]))

In [0]:
%%time
FasttextSearchEngine = SearchFasttext(docs, "./fasttext/model.model")

CPU times: user 1min 52s, sys: 3.48 s, total: 1min 56s
Wall time: 1min 56s


In [0]:
for result in FasttextSearchEngine.search("рождественские каникулы", n=10):
    print(result)

('вечеринки', 7.227940226714265)
('как празднуется Рождество', 6.348252635695177)
('сколько учеников ежегодно посещают весенние каникулы', 6.208211912807602)
('почему атеисты празднуют Рождество', 5.929559275163776)
('Законопроекты', 5.816046160192412)
('программа стипендий выпускников университетов', 5.815099947238545)
('как мусульмане празднуют Рождество', 5.813684691519306)
('какие праздники празднуют атеисты', 5.528922709816639)
('каков ваш обзор каникул', 5.251107693457673)
('почему вы празднуете рождество', 5.220994990144241)


In [0]:
accuracy(FasttextSearchEngine, queries, dup_matrix)

0.08765473402475744

##ELMO

In [0]:
!mkdir elmo
!wget http://vectors.nlpl.eu/repository/11/196.zip -O elmo.zip
!wget https://github.com/ltgoslo/simple_elmo/archive/master.zip
!unzip elmo.zip -d elmo
!unzip master.zip
!mv ./simple_elmo-master/* .
!gzip ./elmo/vocab.txt
!rm *.zip get_elmo_vectors.py LICENSE README.md requirements.txt test.txt vocabulary.py ./simple_elmo-master/.gitignore
!rmdir simple_elmo-master
clear_output()

In [12]:
!ls -R

.:
bilm  elmo  elmo_helpers.py  quora_question_pairs_rus.csv  sample_data

./bilm:
data.py  elmo.py  __init__.py  model.py

./elmo:
meta.json  model.hdf5  options.json  README  vocab.txt.gz

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [0]:
from elmo_helpers import load_elmo_embeddings, get_elmo_vectors
from bilm import Batcher, BidirectionalLanguageModel, weight_layers

In [0]:
class SearchELMO():
    def __init__(self, data, model_path):
        self.texts = np.array(data)
        lemmas = [lemmatize(text) for text in self.texts]
        self.vec = np.zeros((0, 1024))
        self.batcher, self.ids, self.input = load_elmo_embeddings(model_path)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        for i in range(0, len(self.texts), 300):
            self.vec = np.vstack((self.vec,
                                 np.mean(get_elmo_vectors(self.sess,
                                                          lemmas[i: i + 300],
                                                          self.batcher,
                                                          self.ids,
                                                          self.input),
                                         axis=1)))
        self.sess.close()

    def search(self, query, n=5):
        self.sess.run(tf.global_variables_initializer())
        query_vec = np.transpose(np.mean(get_elmo_vectors(self.sess,
                                                          lemmatize(query),
                                                          self.batcher,
                                                          self.ids,
                                                          self.input), axis=1))
        self.sess.close()
        result = np.matmul(self.vec, query_vec)
        indices = np.argsort(result)[::-1].tolist()[:n]
        return list(zip(self.texts[indices], result[indices]))

In [16]:
%%time
ELMOearchEngine = SearchELMO(docs, "./elmo")
clear_output()





Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Sentences in this batch: 300


TypeError: ignored